In [3]:
import pandas as pd
import numpy as np
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from Trax.Cloud.Services.Connector.Logger import LoggerInitializer
from Trax.Utils.Conf.Configuration import Config
from Trax.Data.Projects.Connector import ProjectConnector
from Trax.Cloud.Services.Connector.Keys import DbUsers
from Trax.Cloud.Services.Connector.Factory import BigQueryFactory


LoggerInitializer.init('Pricing Accuracy Analysis')
Config.set_access_mode(Config.CROSS_CLOUD_ACCESS_MODE)
#Config.set_env_and_cloud(Config.PROD, Config.AWS)
Config.set_env_and_cloud(Config.PROD, Config.GCP)

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

2021-11-29 14:00:24,955 - Pricing Accuracy Analysis - 3509041 - WARNING - Logger init method is called more than once. Current app:Pricing Accuracy Analysis, new app: Pricing Accuracy Analysis 


In [4]:
bq_client = BigQueryFactory.get_bigquery_client('trax-ortal-prod')

In [ ]:
sql_probes = """ SELECT *
                        from (SELECT timestamp,event_name,session_uid,wave_type,wave_uid,
                                    project_name,probe_id,data_subject,flavor,accuracy_logic_type,
                                    fr.masking_box,fr.majority_product_fk,
                                    fr.voting_result,fr.voting_result_linkage, fr.majority_value,
                                    fr.majority_is_promotion, fr.production_tag_source_group,
                                    fr.production_tag_source, fr.production_tag_identified,
                                    fr.production_product_fk, fr.TP_value, fr.FP_value, fr.FN_value,
                                    fr.INCONCLUSIVE_value, fr.TP_promotion,fr.FP_promotion,fr.FN_promotion,
                                    fr.TP_linkage,fr.FP_linkage,fr.FN_linkage,fr.INCONCLUSIVE_linkage,
                                    dense_rank() over(partition by event_name, project_name, probe_id, 
                                                                   fr.production_tag_source_group order by timestamp desc) as rownum
                                FROM `trax-ortal-prod.raw.factory_accuracy_price`
                                join unnest (additional_event_data) fr
                                WHERE DATE(timestamp) > "2021-11-01")
                    where rownum = 1
                    """
df = bq_client.run_query(sql_probes).to_dataframe()

In [ ]:
df.head()

In [ ]:
df.accuracy_logic_type.unique()

In [ ]:
df.data_subject.unique()

In [ ]:
df.flavor.unique()

In [ ]:
df.wave_type.unique()

In [ ]:
t1 = df.groupby(['timestamp','project_name','probe_id','majority_product_fk']).agg({'voting_result':'count'}).reset_index()

In [ ]:
t1[t1['voting_result']>1]

In [ ]:
#df[(df['probe_id']==38552) & (df['project_name']=='danonear')&(df['majority_product_fk']==349.0)]
df[(df['probe_id']==38552) & (df['project_name']=='danonear')]

In [ ]:
df.TP_linkage.sum()

In [ ]:
t0=df.groupby(['majority_is_promotion','majority_product_fk']).agg({'masking_box':'count'}).reset_index()

In [ ]:
t3 = df[(df['majority_is_promotion']==1)]
t3.probe_id.count()

In [ ]:
t3.TP_linkage.sum()

In [ ]:
t3.TP_promotion.sum()

In [ ]:
t1 = df[(df['majority_is_promotion']==1)&(df['majority_product_fk'].isnull())]
t1.probe_id.count()

In [ ]:
t2 = df[(df['majority_is_promotion']==1)] 
t2.head()

In [ ]:
df[(df['probe_id']==8731043) & (df['project_name']=='diageoru')]

In [ ]:
sql = """ select *
from trax-ortal-prod.thelake.rds_product
"""
rds_product = bq_client.run_query(sql).to_dataframe()

In [ ]:
rds_product.head()

In [ ]:
rds_product[(rds_product['project_name']=='danonear')&(rds_product['product_pk']==349.0)] #5510

In [ ]:
sql_reg = """select * 
        from
        (select
                event_name as recognition_event_name,
                timestamp as recognition_event_timestamp, 
                wave_type as recognition_wave_type,
                project_name,
                probe_id,
                probe_creation_time, 
                data_subject,
                if(data_subject = 'Price', true, false) as is_price_recognition_probe,
                e.tag_source_group,    
                e.tag_source,
                e.product_fk as product_pk,
                e.facings,--not same with sku level
                e.total_price_tags,
                e.price_value,
                e.is_promotion,
                dense_rank() over(partition by event_name, project_name, probe_id, e.tag_source_group order by timestamp desc) as rownum
            from `trax-ortal-prod.raw.factory_recognition`
            cross join unnest(additional_event_data) as e
            where 1=1
                and date(timestamp) > '2021-10-01'
                and wave_type in ('primary' , 'offline_pricing', 'category_expert')
                and e.tag_source_group in ('Engine', 'QAT') 
                and data_subject = 'Price'
                and flavor = 'default'
        ) 
        where 1=1
        and rownum = 1"""

In [ ]:
df_recognition = bq_client.run_query(sql_reg).to_dataframe()

In [ ]:
df_recognition.head()

In [ ]:
df_recognition[(df_recognition['probe_id']==664187) & (df_recognition['project_name']=='jnjanz')&(df_recognition['product_pk']==5510.0)]

In [ ]:
sql_probes = """ SELECT *
                        from (SELECT timestamp,event_name,session_uid,wave_type,wave_uid,
                                    project_name,probe_id,data_subject,flavor,accuracy_logic_type,
                                    fr.masking_box,fr.majority_product_fk,
                                    fr.voting_result,fr.voting_result_linkage, fr.majority_value,
                                    fr.majority_is_promotion, fr.production_tag_source_group,
                                    fr.production_tag_source, fr.production_tag_identified,
                                    fr.production_product_fk, fr.TP_value, fr.FP_value, fr.FN_value,
                                    fr.INCONCLUSIVE_value, fr.TP_promotion,fr.FP_promotion,fr.FN_promotion,
                                    fr.TP_linkage,fr.FP_linkage,fr.FN_linkage,fr.INCONCLUSIVE_linkage,
                                    dense_rank() over(partition by event_name, project_name, probe_id, 
                                                                   fr.production_tag_source_group order by timestamp desc) as rownum
                                FROM `trax-ortal-prod.raw.factory_accuracy_price`
                                join unnest (additional_event_data) fr
                                WHERE DATE(timestamp) > "2021-11-01")
                    where rownum = 1
                    """
df = bq_client.run_query(sql_probes).to_dataframe()

In [ ]:
fr_query = """
SELECT * FROM (
SELECT probe_id, wave_type, data_subject, session_uid, event_name, timestamp, project_name, 
additional_event_data.tag_source, additional_event_data.facings, additional_event_data.product_fk,
 additional_event_data.tag_source_group, additional_event_data.price_value, additional_event_data.is_promotion,
  flavor, dense_rank() over(partition by event_name, project_name, probe_id, 
                                                                   additional_event_data.tag_source_group order by timestamp desc) as rownum
   FROM `trax-ortal-prod.raw.factory_recognition`
   JOIN UNNEST(additional_event_data) additional_event_data
   WHERE DATE(timestamp) > "2021-11-01")
where 1=1
and rownum=1
LIMIT 5000

"""

fr = bq_client.run_query(fr_query).to_dataframe()

In [ ]:
fr.shape

In [ ]:
df.head()

In [ ]:
fn_link = df[df.FN_linkage == 1] 
fn_link.shape

In [ ]:
fp_link = df[df.FP_linkage == 1]
fp_link.shape

In [ ]:
key_columns = ['probe_id', 'data_subject', 'event_name', 'project_name', 'flavor']

In [ ]:
fn_merge = fn_link.merge(fr, on=key_columns, how='outer')

In [ ]:
fn_merge.shape

In [ ]:
fn_merge

In [ ]:
fn_merge.columns

In [ ]:
x = fn_merge[['majority_value', 'voting_result', 'product_fk', 'majority_product_fk', 'production_product_fk']]

In [ ]:
x[x.product_fk.notna()].sum()

In [ ]:
df.production_tag_source_group.unique()

In [ ]:
df.columns

In [ ]:
fr.tag_source_group.unique()

## JOIN BETWEEN factory_recognition and factory_accuracy_pricing

In [ ]:
query = """

SELECT * FROM
(
SELECT *
            FROM (SELECT timestamp,event_name,session_uid,wave_type,wave_uid,
                        project_name,probe_id,data_subject,flavor,accuracy_logic_type,
                        data.masking_box,data.majority_product_fk,
                        data.voting_result,data.voting_result_linkage, data.majority_value,
                        data.majority_is_promotion, data.production_tag_source_group,
                        data.production_tag_source, data.production_tag_identified,
                        data.production_product_fk, data.TP_value, data.FP_value, data.FN_value,
                        data.INCONCLUSIVE_value, data.TP_promotion,data.FP_promotion,data.FN_promotion,
                        data.TP_linkage,data.FP_linkage,data.FN_linkage,data.INCONCLUSIVE_linkage,
                        dense_rank() over(partition by event_name, project_name, probe_id, 
                                                        data.production_tag_source_group order by timestamp desc) as rownum
                    FROM `trax-ortal-prod.raw.factory_accuracy_price`
                    join unnest (additional_event_data) data
                    WHERE DATE(timestamp) > "2021-11-15")
        where rownum = 1
) fp

full outer join

(
SELECT * FROM (
    SELECT 
        probe_id, wave_type, data_subject, session_uid, event_name, timestamp, project_name, 
        additional_event_data.tag_source, additional_event_data.facings, additional_event_data.product_fk,
        additional_event_data.tag_source_group, additional_event_data.price_value, additional_event_data.is_promotion,
        flavor, dense_rank() over(partition by event_name, project_name, probe_id, 
        additional_event_data.tag_source_group order by timestamp desc) as rownum
    FROM `trax-ortal-prod.raw.factory_recognition`
    JOIN UNNEST(additional_event_data) additional_event_data
    WHERE DATE(timestamp) > "2021-11-15")
    where 1=1
                and date(timestamp) > '2021-11-15'
                and wave_type in ('primary' , 'offline_pricing', 'category_expert')
                and tag_source_group in ('Engine', 'QAT') 
                and flavor = 'default'
                and rownum = 1
) fr

on 1=1
and fr.probe_id = fp.probe_id
and fr.project_name = fp.project_name
and fr.tag_source_group = fp.production_tag_source_group
and fr.product_fk = fp.production_product_fk
;
"""

In [ ]:
joined = bq_client.query_to_dataframe(query)
joined.head()

## JOIN USING factory_recognition fr and factory_accuracy_pricing fap VIEWS

In [5]:
query = """
SELECT
    fap.project_name, fap.probe_id, fap.voting_result_linkage, fap.voting_result, 
    fap.majority_value, fr.price_value, fap.FN_value,
    fap.majority_product_fk, fap.production_product_fk, fr.product_fk, fap.FN_linkage, 
    fr.is_promotion, fap.majority_is_promotion, fap.production_is_promotion, fap.FN_promotion,  
FROM `trax-ortal-prod.sandbox.fap` fap
full outer join  `trax-ortal-prod.sandbox.fr` fr
ON  1=1
    AND fr.probe_id = fap.probe_id
    AND fr.project_name = fap.project_name
    AND fr.tag_source_group = fap.production_tag_source_group 
WHERE 1=1
    OR fap.FN_linkage = 1
    OR fap.FN_promotion = 1
    OR fap.FN_value = 1
LIMIT 10000
"""

df = bq_client.query_to_dataframe(query)

2021-11-29 14:00:45,688 - Pricing Accuracy Analysis - 3509041 - INFO - Run query with job_id=c1c772a0-c45c-42a2-9fb4-a0b40800065d 
2021-11-29 14:00:54,879 - Pricing Accuracy Analysis - 3509041 - INFO - Query job=c1c772a0-c45c-42a2-9fb4-a0b40800065d finished {'total_mb_billed': 2136, 'total_mb_processed': 2136}


In [6]:
df.head()

,project_name,probe_id,voting_result_linkage,voting_result,majority_value,price_value,FN_value,majority_product_fk,production_product_fk,product_fk,FN_linkage,is_promotion,majority_is_promotion,production_is_promotion,FN_promotion
0,heinzcr,4188325,inconclusive,majority,1890.0,NaN,0,NaN,8502.0,NaN,0,None,0.0,0.0,0
1,heinzcr,4188325,inconclusive,majority,1000.0,NaN,0,NaN,8679.0,NaN,0,None,0.0,0.0,0
2,heinzcr,4188325,inconclusive,majority,2300.0,NaN,0,NaN,8675.0,NaN,0,None,0.0,0.0,0
3,heinzcr,4188325,inconclusive,majority,3750.0,NaN,0,NaN,407.0,NaN,0,None,0.0,0.0,0
4,heinzcr,4188325,inconclusive,majority,1280.0,NaN,0,NaN,2972.0,NaN,0,None,0.0,0.0,0


In [7]:
df[(df.FN_linkage == 1) | (df.FN_value == 1) | (df.FN_promotion == 1)]

,project_name,probe_id,voting_result_linkage,voting_result,majority_value,price_value,FN_value,majority_product_fk,production_product_fk,product_fk,FN_linkage,is_promotion,majority_is_promotion,production_is_promotion,FN_promotion
72,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
73,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
74,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
75,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
76,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
77,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
78,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
79,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
80,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
81,pepsicofr,98,inconclusive,majority,NaN,NaN,1,NaN,NaN,NaN,0,None,0.0,NaN,1
